# Some notes on the KZ and LZ experiments, and related experiments

## Where to learn more

We have done our best with documentation!
[docs](https://docs.dwavequantum.com/en/latest/index.html)

We are open source:
[github](https://github.com/dwavesystems/)

Feedback on errors and omissions is welcome: [community](https://github.com/dwavesystems/)

## Embedding:

Minorminer has a variety of heuristic tools, and tools with guaranteed performance:

* general mappings: [minorminer.find_embeddings](https://docs.dwavequantum.com/en/latest/ocean/api_ref_system/generated/minorminer.find_embedding.html#minorminer-find-embedding)
* 1:1 mappings (native/subgraph): [minorminer.subgraph.find_embedding](https://docs.dwavequantum.com/en/latest/ocean/api_ref_minorminer/source/subgraph.html#minorminer.subgraph.find_embedding)
* Clique and Biclique embedding: [minorminer.busclique.find_clique_embedding](https://docs.dwavequantum.com/en/latest/ocean/api_ref_minorminer/source/clique_embedding.html#minorminer.busclique.find_clique_embedding)

Choose the right tool, it can be very inefficient to make a bad choice. Consider a customized approach for regular cases, e.g.:
* Defect-free 1:1 (Next-neighbor-square, Kings, square, triangle) or Defect-free 1:several cubic [hybrid.decomposers.make_origin_embeddings](https://docs.dwavequantum.com/en/latest/ocean/api_ref_hybrid/generated/hybrid.decomposers.make_origin_embeddings.html#hybrid.decomposers.make_origin_embeddings):

Parallelization of embeddings recently supported:
[minorminer.utils.parallel_embedding.find_multiple_embeddings](https://docs.dwavequantum.com/en/latest/ocean/api_ref_minorminer/source/clique_embedding.html#minorminer.utils.parallel_embeddings.find_multiple_embeddings)

## Noise-mitigation/investigation:
Most of our research papers involve shimming. The general-purpose calibration can be refined for specific models to eliminate specific cross-talks and non-idealities:
* [Calibration refinement](https://doi.org/10.3389/fcomp.2023.1238988) Published paper and (recently updated) demonstration code; We can tune parameters such as flux_biases to restore expected symmetries.
* We can extend this to learning general distributions [pytorch plug-in](https://github.com/dwavesystems/dwave-pytorch-plugin/blob/main/README.rst)

Another option is to use spin-reversal transforms: 
* Spin-reversal transforms with respect to the noise-free evolution amounts to a change of basis (gauge)
* With respect to the programmed realization, a change of basis leads to new noise patterns.
* By averaging we eliminate noise. e.g. the effect of independent and identically distributed flux (h) noise is eliminated at leading order.
```
    qpu = DWaveSampler(solver='Advantage2_prototype2.6')
    sampler = SpinReversalTransformComposite(qpu)  # Composites applies (and inverts) the transformation
```

Another option is averaging over embeddings:
* Use randomized or parallelized embeddings
```
    qpu = DWaveSampler(solver='Advantage2_prototype2.6')
    sampler = SpinReversalTransformComposite(qpu)  
    source = nx.from_edgelist([(i, i+1) for i in range(3)])  # A path (1d chain) of size 4
    sampler = ParallelEmbeddingComposite(sampler) # Composites distributes and inverts (a distribution on 4-spins is returned)
```
* Exploit symmetries of the source model (Ising model) to permute a known embedding: 

## Temperature (quasi-static freezeout)

If the model is maximum entropy, subject to some rate of excitations (energy), it is a Boltzmann distribution.

The local pattern of excitations is sufficient to infer the temperature with enough samples:
```
    qpu = DWaveSampler(solver='Advantage2_prototype2.6')
    dwave.system.temperatures.fast_effective_temperature(qpu)  # By default, uses independent spins
```
Note that, if the distribution is not Boltzmann due to ergodicity breaking (trapping by local minima) we expect the rate of local excitations to be smaller than the rate of general excitations. Quantum-Boltzmann and noise effects can also lead to bias in estimators.

## Jupyter notebook syntax (for reference, delete later)

This is some plain text that forms a paragraph. Add emphasis via **bold** or __bold__, and *italic* or _italic_.

Paragraphs must be separated by an empty line.

* Sometimes we want to include lists.
* Which can be bulleted using asterisks.

1. Lists can also be numbered.
2. If we want an ordered list.

[It is possible to include hyperlinks](https://www.dataquest.io)

Inline code uses single backticks: `foo()`, and code blocks use triple backticks:
```
bar()
```
Or can be indented by 4 spaces:
```
    foo()
```

And finally, adding images is easy: ![Alt text](https://www.dataquest.io/wp-content/uploads/2023/02/DQ-Logo.svg)
